In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/IMSAI/image.zip -d /content/sample_data/

Archive:  /content/drive/MyDrive/IMSAI/image.zip
  inflating: /content/sample_data/image/o_1.png  
  inflating: /content/sample_data/image/o_10.png  
  inflating: /content/sample_data/image/o_100.png  
  inflating: /content/sample_data/image/o_101.jpg  
  inflating: /content/sample_data/image/o_102.jpg  
  inflating: /content/sample_data/image/o_103.jpg  
  inflating: /content/sample_data/image/o_104.jpg  
  inflating: /content/sample_data/image/o_105.jpg  
  inflating: /content/sample_data/image/o_106.jpg  
  inflating: /content/sample_data/image/o_107.jpg  
  inflating: /content/sample_data/image/o_108.jpg  
  inflating: /content/sample_data/image/o_109.jpg  
  inflating: /content/sample_data/image/o_11.png  
  inflating: /content/sample_data/image/o_110.jpg  
  inflating: /content/sample_data/image/o_111.jpg  
  inflating: /content/sample_data/image/o_112.jpg  
  inflating: /content/sample_data/image/o_113.jpg  
  inflating: /content/sample_data/image/o_114.jpg  
  inflating: /conte

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

path = r"/content/sample_data/image"
def findFiles(path): return glob.glob(path)
#데이터 불러오기

In [ ]:
num_epochs = 15  # 에폭 5 10 15 20

In [6]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import os
import cv2
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import random

# 랜덤 시드 설정 함수
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 사용 여부 설정
use_seed = False  # True로 설정하면 랜덤 시드를 고정, False로 설정하면 랜덤 시드 사용 안함
seed = 42

if use_seed:
    set_seed(seed)

# 이미지 파일 리스트 생성
image_list1 = [os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith('.png')]
image_list2 = [os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith('.jpg')]

image_list = image_list1 + image_list2
transform = transforms.ToTensor()

dataset = []
labels = []



# 이미지 전처리
for idx in image_list:
    image = cv2.imread(idx, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (150, 150))
    image = Image.fromarray(image)
    image = transform(image)
    dataset.append(image)

    filename = os.path.basename(idx).lower()
    if 'o' in filename:
        labels.append(0)  # 결과 레이블 'o'
    elif 'x' in filename:
        labels.append(1)  # 결과 레이블 'x'

# 리스트를 텐서로 변환
labels = torch.tensor(labels, dtype=torch.long)

# 데이터셋 분할
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(list(zip(dataset, labels)), [train_size, test_size])


batch_size = 100  # 배치 사이즈 60(69) 64(75) 85(76) 100(71) 여기 다시
learning_rate = 0.0001  # 러닝 레이트 0.01(57),0.001(75) ,0.0005(75)), 0.0001(66)
activation_function = nn.ReLU  # 활성화 함수
num_hidden_layers = 5  # 히든레이어 계층수 (출력층 포함) 2(69), 3(75), 4(76), 5(71)
padding = 1  # 패딩 0 2 3
stride = 1  # 스트라이드 2 3 4
kernel_size = 3  # 커널 사이즈  3, 5, 7, 9
optimizer_function = optim.Adam  # 최적화 함수
channels = [1, 32, 256] # 채널 크기 [1, 4, 36](69), [1, 8, 64](67), [1, 16, 128](76), [1, 32, 256](73)
normalization = nn.BatchNorm2d  # 정규화
num_epochs = 20



lists = [20,21,22,23,24]
for i in lists:
  print("num_epochs")
  print(i)
  num_epochs = i


  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


  class CNN(nn.Module):
      def __init__(self):
          super(CNN, self).__init__()
          self.conv1 = nn.Sequential(
              nn.Conv2d(channels[0], channels[1], kernel_size=kernel_size, stride=stride, padding=padding),
              normalization(channels[1]),
              activation_function(),
              nn.MaxPool2d(2))

          self.conv2 = nn.Sequential(
              nn.Conv2d(channels[1], channels[2], kernel_size=kernel_size, stride=stride, padding=padding),
              normalization(channels[2]),
              activation_function(),
              nn.MaxPool2d(2))

          self.fc = nn.Linear(channels[2] * 37 * 37, 2)

      def forward(self, x):
          x = self.conv1(x)
          x = self.conv2(x)
          x = x.view(x.size(0), -1)
          x = self.fc(x)
          return x

  model = CNN()
  optimizer = optimizer_function(model.parameters(), lr=learning_rate)
  criterion = nn.CrossEntropyLoss()

  # 모델 학습
  for epoch in range(num_epochs):
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          inputs, labels = data
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

  # 모델 성능 평가
  correct = 0
  total = 0
  with torch.no_grad():
      for data in test_loader:
          inputs, labels = data
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Result: {100 * correct / total}%')


num_epochs
20
Epoch [1/20], Loss: 1.7158
Epoch [2/20], Loss: 2.0007
Epoch [3/20], Loss: 1.8403
Epoch [4/20], Loss: 0.4967
Epoch [5/20], Loss: 0.6745
Epoch [6/20], Loss: 0.1383
Epoch [7/20], Loss: 0.4587
Epoch [8/20], Loss: 0.2191
Epoch [9/20], Loss: 0.0140
Epoch [10/20], Loss: 0.0873
Epoch [11/20], Loss: 0.0804
Epoch [12/20], Loss: 0.0097
Epoch [13/20], Loss: 0.0027
Epoch [14/20], Loss: 0.0268
Epoch [15/20], Loss: 0.0180
Epoch [16/20], Loss: 0.0041
Epoch [17/20], Loss: 0.0020
Epoch [18/20], Loss: 0.0061
Epoch [19/20], Loss: 0.0012
Epoch [20/20], Loss: 0.0011
Result: 82.14285714285714%
num_epochs
21
Epoch [1/21], Loss: 4.2688
Epoch [2/21], Loss: 3.0362
Epoch [3/21], Loss: 1.4775
Epoch [4/21], Loss: 1.7830
Epoch [5/21], Loss: 0.7259
Epoch [6/21], Loss: 0.5064
Epoch [7/21], Loss: 0.5726
Epoch [8/21], Loss: 0.1407
Epoch [9/21], Loss: 0.2048
Epoch [10/21], Loss: 0.1551
Epoch [11/21], Loss: 0.0328
Epoch [12/21], Loss: 0.0257
Epoch [13/21], Loss: 0.0428
Epoch [14/21], Loss: 0.0167


KeyboardInterrupt: 